<a href="https://colab.research.google.com/github/as9786/ML-DLPratice/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/%EC%9D%B4%EB%AF%B8%EC%A7%80%EB%82%B4%EB%B6%80%EC%9D%98%EB%AC%B8%EC%9E%90%EC%9D%B8%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 손글씨 숫자 인식하기

## MNIST 데이터

In [1]:
import urllib.request as req
import gzip,os,os.path

In [11]:
savepath = '/content/drive/MyDrive/머신러닝,딥러닝실전/4장/mnist'
baseurl = 'http://yann.lecun.com/exdb/mnist/'
files = ['t10k-images-idx3-ubyte.gz','t10k-labels-idx1-ubyte.gz','train-images-idx3-ubyte.gz','train-labels-idx1-ubyte.gz']

In [19]:
# 다운로드
if not os.path.exists(savepath):os.mkdir(savepath)
for f in files:
    url = baseurl+f
    loc = savepath+'/'+f
    print('download:',url)
    if not os.path.exists(loc):
        req.urlretrieve(url,loc)

download: http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
download: http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz


In [20]:
# Gzip 압축 해제
for f in files:
    gz_file = savepath+'/'+f
    raw_file = savepath+'/'+f.replace('.gz','')
    print('gzip:',f)
    with gzip.open(gz_file,'rb') as fp:
        body = fp.read()
        with open(raw_file,'wb') as w:
            w.write(body)
print('ok')

gzip: t10k-images-idx3-ubyte.gz
gzip: t10k-labels-idx1-ubyte.gz
gzip: train-images-idx3-ubyte.gz
gzip: train-labels-idx1-ubyte.gz
ok


In [39]:
%cd /content/drive/MyDrive/머신러닝,딥러닝실전/4장

/content/drive/MyDrive/머신러닝,딥러닝실전/4장


In [42]:
import struct

def to_csv(name,maxdata):
    # 레이블 파일과 이미지 파일 열기
    lbl_f = open('./mnist/'+name+'-labels-idx1-ubyte','rb')
    img_f = open('./mnist/'+name+'-images-idx3-ubyte','rb')
    csv_f = open('./mnist/'+name+'.csv','w',encoding='utf-8')
    #헤더 정보 읽기
    mag, lbl_count = struct.unpack('>II',lbl_f.read(8))
    mag, img_count = struct.unpack('>II',img_f.read(8))
    rows,cols = struct.unpack('>II',img_f.read(8))
    pixels = rows * cols
    #이미지 데이터를 읽고 csv로 저장
    for idx in range(lbl_count):
        if idx > maxdata: break
        label = struct.unpack('B',lbl_f.read(1))[0]
        bdata = img_f.read(pixels)
        sdata = list(map(lambda n : str(n),bdata))
        csv_f.write(str(label)+',')
        csv_f.write(','.join(sdata)+'\r\n')

        # 잘 저장됐는지 이미지 파일로 저장해서 테스트
        if idx < 10:
            s = 'P2 28 28 255\n'
            s += ' '.join(sdata)
            iname = './mnist/{0}-{1}-{2}.pgm'.format(name,idx,label)
            with open(iname,'w',encoding='utf-8') as f:
                f.write(s)
    csv_f.close()
    lbl_f.close()
    img_f.close()

In [44]:
to_csv("train", 1000)
to_csv('t10k',500)

# 이미지 데이터 학습시키기

In [45]:
from sklearn import model_selection, svm, metrics

In [46]:
def load_csv(fname):
    labels = []
    images = []

    with open(fname,'r') as f:
        for line in f:
            cols = line.split(',')
            if len(cols) < 2: continue
            labels.append(int(cols.pop(0)))
            vals = list(map(lambda n:int(n)/256,cols))
            images.append(vals)
    return {'labels':labels,'images':images}

In [47]:
data = load_csv('./mnist/train.csv')
test = load_csv('./mnist/t10k.csv')

In [48]:
clf = svm.SVC()
clf.fit(data['images'],data['labels'])

SVC()

In [53]:
predict = clf.predict(test['images'])

In [54]:
ac_score = metrics.accuracy_score(test['labels'],predict)
cl_report = metrics.classification_report(test['labels'],predict)
print('정답률=',ac_score)
print('리포트=')
print(cl_report)

정답률= 0.8842315369261478
리포트=
              precision    recall  f1-score   support

           0       0.87      0.98      0.92        42
           1       0.99      1.00      0.99        67
           2       0.91      0.89      0.90        55
           3       0.94      0.72      0.81        46
           4       0.86      0.93      0.89        55
           5       0.75      0.82      0.78        50
           6       0.95      0.81      0.88        43
           7       0.79      0.94      0.86        49
           8       0.94      0.82      0.88        40
           9       0.89      0.87      0.88        54

    accuracy                           0.88       501
   macro avg       0.89      0.88      0.88       501
weighted avg       0.89      0.88      0.88       501



## 6만 개의 모든 데이터 학습시키기

In [55]:
to_csv("train", 99999)

In [56]:
def load_csv(fname):
    labels = []
    images = []

    with open(fname,'r') as f:
        for line in f:
            cols = line.split(',')
            if len(cols) < 2: continue
            labels.append(int(cols.pop(0)))
            vals = list(map(lambda n:int(n)/256,cols))
            images.append(vals)
    return {'labels':labels,'images':images}

In [57]:
data = load_csv('./mnist/train.csv')
test = load_csv('./mnist/t10k.csv')

In [58]:
clf = svm.SVC()
clf.fit(data['images'],data['labels'])

SVC()

In [59]:
predict = clf.predict(test['images'])

In [60]:
ac_score = metrics.accuracy_score(test['labels'],predict)
cl_report = metrics.classification_report(test['labels'],predict)
print('정답률=',ac_score)
print('리포트=')
print(cl_report)

정답률= 0.9720558882235529
리포트=
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        42
           1       1.00      1.00      1.00        67
           2       0.96      0.96      0.96        55
           3       0.98      0.98      0.98        46
           4       0.98      0.96      0.97        55
           5       1.00      0.96      0.98        50
           6       0.98      0.95      0.96        43
           7       0.96      0.98      0.97        49
           8       0.93      0.97      0.95        40
           9       0.96      0.94      0.95        54

    accuracy                           0.97       501
   macro avg       0.97      0.97      0.97       501
weighted avg       0.97      0.97      0.97       501

